# Data downloading

In [ ]:
# !pip install hda
# !pip install rasterio==1.3.0
# !pip install python-dotenv
# !pip install sentinelsat
# !pip install pandas
# !pip install geopandas
# !pip install folium

In [1]:
import hda
from pathlib import Path
from hda import Client
import os

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

from shapely import wkt
import geopandas as gpd
import rasterio as rio
from pyproj import Proj
from pyproj import Transformer
from rasterio.mask import mask
from shapely.geometry import Polygon
from rasterio.warp import calculate_default_transform, reproject
import zipfile
import numpy as np
from skimage.transform import resize
import shutil
import pickle
import zarr
from glob import glob

#User = 
#Password = 
#Token = 

In [2]:
from pyproj import CRS

## Search of products

In [3]:
# Define range of interest

In [3]:
api = SentinelAPI(User, Password,'https://apihub.copernicus.eu/apihub')
first_date = date(2018, 12, 15)
last_date = date(2018, 12, 31)

footprint = "POLYGON((-70.9649357878204 -33.81244707234685,-70.96591676385808 -33.89260670554516,-70.84251675089554 -33.89488656955273,-70.84506715448308 -33.81375110356652,-70.9649357878204 -33.81244707234685))"

products = api.query(footprint,
                     date=(first_date, last_date),
                         platformname='Sentinel-2',
                     processinglevel='Level-1C')#,
                     #cloudcoverpercentage=(0, 100))

print(f'{len(products)} products found')

4 products found


## Extract area of interest in original CRS

In [4]:
initial_geometry = [wkt.loads(footprint)]

In [5]:
c = Client(url='https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker',
           user = User,
           password = Password,
           token=Token, debug=False, quiet=True);

In [8]:
# Methods
def area2ts(p):
    n = p['filename'].split('.')[0]
    datstrip = p['datastripidentifier'].split('_')[8][1:]
    granuleid = p['granuleidentifier'].split('_')



    identifier = p['identifier']
    request = { "datasetId": "EO:ESA:DAT:SENTINEL-2:MSI",
               "stringInputValues": [{"name": "productIdentifier", "value": identifier}]}

    #matches = c.search(request)
    title = p['title']
    #matches.download()

    filepath = title + '.zip'
    with zipfile.ZipFile(filepath,"r") as zip_ref:
        zip_ref.extractall("../data")
    os.remove(filepath)


    source_crs = "EPSG:4326"
    
    fullband = []

    granule_folder = glob("../data/{}.SAFE/GRANULE/*/".format(n), recursive = True)[0]
    
    for band in ['B01','B02','B03','B04','B05','B06','B07','B08','B09','B10','B11','B12','B8A']:
        with rio.open('{}/IMG_DATA/{}_{}_{}.jp2'.format(granule_folder, n.split('_')[5], n.split('_')[2], band)) as img:
            print(img.crs)
            target_crs = img.crs.to_string()
            x, y = initial_geometry[0].exterior.coords.xy

            aoi = list(zip(x, y))
            transformer = Transformer.from_crs(source_crs, target_crs)

            new_coords = []
            for co in aoi:
                t = transformer.transform(co[1],co[0])
                new_coords.append(t)

            aoi = [Polygon(new_coords)]

            clipped, transform = mask(img, aoi, crop=True)
            metadata = img.meta.copy()

            metadata.update({"transform": transform,
                         "height": clipped.shape[1],
                         "width": clipped.shape[2]#,
                            #'driver': 'GTiff'
                            })

            with rio.open('../temp/{}_{}.tif'.format(n, band), 'w', **metadata) as dst:
                dst.write(clipped)


            with rio.open('../temp/{}_{}.tif'.format(n,band)) as r:
                fullband.append(r.read())

            os.remove('../temp/{}_{}.tif'.format(n,band))
            os.remove('../temp/{}_{}.tif.aux.xml'.format(n,band)) 

    
    fullband_resized = []
    max_shape = tuple(np.max([np.shape(np.squeeze(band)) for band in fullband],axis=0))
    
    
    for img in fullband:
        if img.shape != max_shape:
            image_resized = resize(np.squeeze(img), max_shape, anti_aliasing=False, preserve_range=True)
            fullband_resized.append(image_resized)
    
    shutil.rmtree('../data/{}.SAFE'.format(n))
    return np.array(fullband_resized, dtype=np.int16)

def load_and_append_zarr(array, filename='output.zarr'):
    if os.path.isdir(filename):
        z = zarr.open(filename, mode='a')
        z.append(array[np.newaxis])
        zarr.save(filename, z)
    else:
        z.save(filename, array[np.newaxis])
                      
def load_and_expand_zarr(array, key, filename='output.zarr', debug=False):
    if os.path.isdir(filename):
        z = zarr.open(filename) #, mode='a')
        z[key] = array
        #zarr.save(filename, z)
    else:
        zarr.save(filename, **{key: array})

In [7]:
rio.__version__

'1.3.0'

In [9]:
area2ts(products['470e2932-6334-4b3d-a272-be23c63a2d1b'])

FileNotFoundError: [Errno 2] No such file or directory: 'S2A_MSIL1C_20181230T143751_N0207_R096_T19HCC_20181230T180118.zip'

In [10]:
pp ='../data/S2B_MSIL1C_20171220T143739_N0206_R096_T19HCC_20171220T175623.SAFE/GRANULE/L1C_T19HCC_A004126_20171220T144655/IMG_DATA/T19HCC_20171220T143739_TCI.jp2'
with rio.open(pp) as img:
    print(img.meta)

{'driver': 'JP2OpenJPEG', 'dtype': 'uint8', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 3, 'crs': None, 'transform': Affine(10.0, 0.0, 300000.0,
       0.0, -10.0, 6300040.0)}


In [2]:
pp = 'T19HCC_20181230T143751_B04.jp2'
with rio.open(pp) as img:
    print(img.meta)

{'driver': 'JP2OpenJPEG', 'dtype': 'uint16', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 300000.0,
       0.0, -10.0, 6300040.0)}


In [15]:
import rasterio
crs = rasterio.crs.CRS({'init': 'epsg:4326'})
print(crs.is_valid)

CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: C:\Users\Carlos\anaconda3\Library\share\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.

In [18]:
from rasterio.crs import CRS
#CRS.from_epsg(32721)
CRS.from_epsg(32634)

CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: C:\Users\Carlos\anaconda3\Library\share\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.

## Download of selected products

In [9]:
## Testing parallel loading of ZARR
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def paral(func, lista, N, threads=True, processes=False):
    if processes:
        with ProcessPoolExecutor(max_workers=N) as executor:
            results = executor.map(func, lista)
        return list(results)
    elif threads:
        with ThreadPoolExecutor(max_workers=N) as executor:
            results = executor.map(func, lista)
        return list(results)

In [10]:
outputZarr = '2017.zarr'
failedProducts = []
for p in products:
    try:
        if os.path.isdir(outputZarr):
            if p not in list(zarr.load(outputZarr)):
                print('{} downloading new dataset'.format(p))
                image = area2ts(products[p])
                load_and_expand_zarr(image, p, filename=outputZarr)
            else:
                print('{} already downloaded and processed'.format(p))
        else:
            print('{} downloading new initial dataset'.format(p))
            image = area2ts(products[p])
            load_and_expand_zarr(image, p, filename=outputZarr)

        
    except Exception as e:
        if  os.path.isfile('{}.zip'.format(products[p]['title'])):
            os.remove('{}.zip'.format(products[p]['title']))
            
        if  os.path.isdir('data/{}.SAFE'.format(products[p]['title'])):
            shutil.rmtree('data/{}.SAFE'.format(products[p]['title']))
            
        failedProducts.append(p)
        print('{} error. Ignoring dataset'.format(p))
        print(e)

4506275d-1f35-4916-abd0-4316e7d577fb downloading new initial dataset


C:\Users\Carlos\anaconda3\envs\kidodynamics\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zipper.creodias.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


4506275d-1f35-4916-abd0-4316e7d577fb error. Ignoring dataset
'NoneType' object has no attribute 'to_string'
2499a745-727d-4f2d-a80a-64d05b21a033 downloading new initial dataset


C:\Users\Carlos\anaconda3\envs\kidodynamics\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zipper.creodias.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
                                                     
KeyboardInterrupt



In [ ]:
# ## output of failed
# with open('{}_failed.txt'.format(outputZarr.split('.')[0]), 'w') as fp:
#     for item in np.unique(failedProducts):
#         # write each item on a new line
#         fp.write("%s\n" % item)

In [ ]:
# ## do dataframe with the information. Basically matches to dataframe
# entries = list(zarr.load(outputZarr))
# df = api.to_dataframe(products)
# df = df.loc[entries]
# df.to_pickle('{}_df.pickle'.format(outputZarr.split('.')[0]))
# df

In [ ]:
# os.system("zip -r {}.zip {} ".format(outputZarr.split('.')[0], outputZarr)) #>/dev/null 2>&1
# #os.system("unzip {}.zip".format(outputZarr.split('.')[0]))